# Klassifikation I

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelerntetn Regeln einordnen. 

In unserem Beispiel nehmen wir Texte, die zu verschiedenen Klassen gehören. Die Texte sind deutschsprachige Abstracts von Dissertationen der TU Berlin; die Klassen sind die Fakultäten, an denen die Dissertationen verfasst wurden.

Zunächst lesen wir alle Texte ein. Als Merkmale für einen Text nehmnen wir die Frequenz (Häufigkeit durch Gesamtzahl der Wörter) der Lemmata der Wörter aus den offenen Klassen. Sicherheitshalber filtern wir auch Stop-wörter heraus.

In [1]:
import codecs
import glob
import nltk
import pprint
import html
import treetaggerwrapper
import collections
from nltk.corpus import stopwords


stopwords = set(stopwords.words('german'))
ttagger = treetaggerwrapper.TreeTagger(TAGLANG='de')

data = []
wordlist = []

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    
    return False

def features_from_text(text):
    tlen = 0
    wordcounts = collections.Counter()
    
    sentences = nltk.sent_tokenize(text)
    for sent in sentences:
        tokens = nltk.tokenize.word_tokenize(sent,language='german')
        tags = ttagger.tag_text(tokens,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        lemmata_sent = [lemma for (word,pos,lemma) in tags2 if not closed_class(pos) and not word in stopwords]
        #lemmata.extend(lemmata_sent)
        tlen += len(lemmata_sent)
        wordcounts.update(lemmata_sent)

    return {w:(float(wordcounts[w])/float(tlen)) for w in wordcounts}


def read_data(verzeichnis):
    data = []
    for v in glob.glob(verzeichnis+"\*"):
        c = v[len(verzeichnis)+1:]
        #print(c)
        for datei in glob.glob(v +"\*.txt"):
            textfile = codecs.open(datei, "r", "latin1")
            text = html.unescape(textfile.read()) #Texte sind nicht ganz sauber gespeichert!
            textfile.close()
            data.append((features_from_text(text),c))
    return data
            
    
train_data = read_data("C:\Data\TUB-Abstracts-Train")

In [2]:
Wir schauen uns jetzt mal ein Dokument an:

SyntaxError: invalid syntax (<ipython-input-2-5e5a4d21e778>, line 1)

In [3]:
train_data[27]

({'Abbild': 0.007633587786259542,
  'Analyse': 0.015267175572519083,
  'Ansatz': 0.015267175572519083,
  'Anteil': 0.007633587786259542,
  'Arbeit': 0.022900763358778626,
  'Autor': 0.007633587786259542,
  'Beispiel': 0.007633587786259542,
  'Beitrag': 0.007633587786259542,
  'Bekanntheitsgrad': 0.007633587786259542,
  'Bild': 0.030534351145038167,
  'Bildhermeneutik': 0.007633587786259542,
  'Bildkompetenz': 0.015267175572519083,
  'Bildmaterial': 0.007633587786259542,
  'Bildquellenkritik': 0.007633587786259542,
  'Bildung': 0.015267175572519083,
  'Dauer': 0.007633587786259542,
  'Deutung': 0.007633587786259542,
  'Einsatz': 0.007633587786259542,
  'Entwicklung': 0.007633587786259542,
  'Formatierung': 0.007633587786259542,
  'Fotografie': 0.022900763358778626,
  'Frage': 0.022900763358778626,
  'Gedächtnis': 0.007633587786259542,
  'Generierung': 0.007633587786259542,
  'Geschichtsbewusstsein': 0.007633587786259542,
  'Geschichtsdidaktik': 0.015267175572519083,
  'Geschichtsunterri

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben dein Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. Wir filtern daher alle Wörter aus die weniger als 5 mal vorkommen. 

In [4]:
docfreq = collections.Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

def filter_rare(data):
    fdata = []
    for (wfreq,c) in data:
        wfreq1 = {}
        for w in wfreq:
            if docfreq[w] >= 5:
                wfreq1[w] = wfreq[w]
        fdata.append((wfreq1,c))
    return fdata

            
train_data = filter_rare(train_data)

In [5]:
train_data[27]

({'Analyse': 0.015267175572519083,
  'Ansatz': 0.015267175572519083,
  'Anteil': 0.007633587786259542,
  'Arbeit': 0.022900763358778626,
  'Autor': 0.007633587786259542,
  'Beispiel': 0.007633587786259542,
  'Beitrag': 0.007633587786259542,
  'Bild': 0.030534351145038167,
  'Bildung': 0.015267175572519083,
  'Deutung': 0.007633587786259542,
  'Einsatz': 0.007633587786259542,
  'Entwicklung': 0.007633587786259542,
  'Frage': 0.022900763358778626,
  'Generierung': 0.007633587786259542,
  'Gesellschaft': 0.007633587786259542,
  'Häufigkeit': 0.007633587786259542,
  'Integration': 0.007633587786259542,
  'Praxis': 0.007633587786259542,
  'Prozess': 0.007633587786259542,
  'Publikation': 0.007633587786259542,
  'Quelle': 0.015267175572519083,
  'Rezeption': 0.007633587786259542,
  'Sinn': 0.007633587786259542,
  'Status': 0.007633587786259542,
  'Synthese': 0.007633587786259542,
  'Thema': 0.015267175572519083,
  'Trennung': 0.007633587786259542,
  'Umgang': 0.007633587786259542,
  'Untersu

## Dokumentähnlichkeit

Wir können jetzt die Ähnlichkeit zwischen zwei Dokumenten berechnen, zum Beispile mit dem Cosinus. Wir erinnern uns:
$$
cos(a,b) = \frac{\sum_i a_i b_i}{\sqrt{\sum_i a_i^2}\sqrt{\sum_i b_i^2}}
$$

In [7]:
import math

def cosinus(a,b):
    return sum([a[k]*b[k] for k in a if k in b]) / (math.sqrt(sum([a[k]**2 for k in a])) * math.sqrt(sum([b[k]**2 for k in b])))

Wir vergleichen jetzt mal die Ähnlichkeit von zwei willkürlichen Dokumenten aus einer Klasse mit der Ähnlickeit von zwei Dokumenten aus unterschiedlichen Klassen.

In [8]:
d1,c1 = train_data[35]
d2,c2 = train_data[36]
d3,c3 = train_data[225]

print(c1,c2,c3)

print(cosinus(d1,d2))
print(cosinus(d1,d3))
print(cosinus(d2,d3))

Fakultät I Fakultät I Fakultät III
0.18748200356602088
0.09935472241731383
0.026246086344164105


## k-Nearest-Neighbors (kNN)

Ein sehr einfaches Klassifikationsverfahren ist das k-Nearest-Neighbors (kNN) oder k-Nächste-Nachbarn-Verfahren.

Im Training machen wir nichts, ausser alle Beispile zu speichern. In der Klassifikation suchn wir die _k_ Nachbarn, die dem zu klassifizierenden Element am ähnlichsten sind. Diesert Schritt ist natürich sehr aufwändig! Die Klasse, die unter diesen Nachbarn am häufigsten Vertreten ist, ist der Vorshlag für das klassifizierende Element.

In [9]:
def kNN(features,k,train):
    similarities = []
    for d,c in train:
        similarities.append((c,cosinus(d,features)))
    similarities = sorted(similarities,key= lambda x:x[1],reverse=True)
    cnt = collections.Counter({c:sim for (c,sim) in similarities[:k]})
    most_likely = cnt.most_common(1)
    return most_likely[0][0]

In [10]:
test_data = read_data("C:\Data\TUB-Abstracts-Test")
test_data = filter_rare(test_data)

Wir versuchen es einfach mal:

In [12]:
f,c = test_data[86]
print("Korrekt wäre:",c)
vorhersage = kNN(f,3,train_data)
print("Vorhersage ist:",vorhersage)

Korrekt wäre: Fakultät V
Vorhersage ist: Fakultät III


Wir testen jetzt mal systematisch alle Testdaten

In [17]:
n = 0
correct = 0
for f,c in test_data:
    p = kNN(f,1,train_data)
    n+= 1
    if p==c:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/float(n)))

42.4 Prozent korrekt


Das ist schon gar nicht so schlecht, wenn auch nicht sehr beeidruckend. 

Was hätten wir mit raten erreicht? Wir würden z.B. immer Fakultät I sagen, und hätten dann $\frac{1}{7}$, als eta 14% korrekt. Da liegen wir deutlich drüber.

Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ise eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [18]:
gold = [c for f,c in test_data]
test = [kNN(f,2,train_data) for f,c in test_data]
cm = nltk.ConfusionMatrix(gold, test)
print(cm) 

             |        F           F |
             |     F  a  F     F  a |
             |  F  a  k  a  F  a  k |
             |  a  k  u  k  a  k  u |
             |  k  u  l  u  k  u  l |
             |  u  l  t  l  u  l  t |
             |  l  t  ä  t  l  t  ä |
             |  t  ä  t  ä  t  ä  t |
             |  ä  t     t  ä  t    |
             |  t     I     t     V |
             |     I  I  I     V  I |
             |  I  I  I  V  V  I  I |
-------------+----------------------+
  Fakultät I | <8> 1  .  2  .  6  3 |
 Fakultät II |  .<12> 3  3  .  2  . |
Fakultät III |  .  4 <8> .  3  4  1 |
 Fakultät IV |  .  6  1 <8> 3  1  1 |
  Fakultät V |  1  2  4  5 <7> 1  . |
 Fakultät VI |  3  3  2  1  1 <9> 1 |
Fakultät VII |  1  .  .  2  3  2 <4>|
-------------+----------------------+
(row = reference; col = test)



## Termgewichtung: tf.idf

Wir berücksichtigen jetzt alle Terme gleich stark. Werden wir besser wenn wir tf.idf Werte statt Frequenzen nutzen?

In [19]:
def tfidf_transform(data):
    D = float(len(data))
    tdata = []
    
    for (wfreq,c) in data:
        tfidf = {}
        for w in wfreq:
            tfidf[w] = wfreq[w] * math.log(D/float(docfreq[w]))
        tdata.append((tfidf,c))
    
    return tdata


In [20]:
train_data_tfidf = tfidf_transform(train_data)
test_data_tfidf = tfidf_transform(test_data)

Wir versuchen es nochmal:

In [55]:
n = 0
correct = 0
for f,c in test_data_tfidf:
    p = kNN(f,1,train_data_tfidf)
    n+= 1
    if p==c:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/float(n)))

48.5 Prozent korrekt


## Nearest Centroid

Ein Verfahren, dass bei der Klassifikation wesentlich effizienter ist, aber nach einem ähnlichem Prizip funktioniert, ist der Nearest Centroid Klassifikator. Wir erstellen jetzt für jede Klasse ein Musterdokument. Dieses Dokument hat für jedes Merkmal die Durchschnittswerte aller Dokumente aus einer Klasse. Für ein neues Dokument suchen wir jetzt das nächste Zentrum.

In [41]:
def get_centers(data):
    centers = {}
    class_sizes = {}
    for featmap,c in data:
        center = centers.get(c,{})
        class_sizes[c] = 1 + class_sizes.get(c,0)
        for ft in featmap:
            freq = featmap[ft]
            center[ft] = freq + center.get(ft,0)
        centers[c] = center
    for cls in centers:
        center = centers[cls]
        for f in center:
            center[f] /= class_sizes[cls]
    return centers

In [42]:
cls_centers = get_centers(train_data_tfidf)

In [40]:
cls_centers

{'Fakultät I': {'Potential': 0.07334750579538193,
  'Einsatz': 0.2596924454527444,
  'neu': 0.5606681705421664,
  'Medium': 0.829632318298013,
  'Beispiel': 0.33461572292054725,
  'Projekt': 0.26021844931409077,
  'lang': 0.10668875127961117,
  '1997': 0.06028888846520684,
  '9': 0.06015846367653252,
  'häufig': 0.1883080493392598,
  'Rede': 0.08714634824079795,
  'deutsch': 0.7517939731368598,
  'sprechen': 0.08717500231921768,
  'strukturell': 0.13406284011603556,
  'Veränderung': 0.19998088343447648,
  'stehen': 0.314398818287028,
  'selten': 0.07184319995525756,
  'ebenfalls': 0.064747384612807,
  'Versuch': 0.13340758289657934,
  'stark': 0.14518180285293292,
  'einbinden': 0.031963066197527805,
  'zahlreich': 0.19061915142393965,
  'demonstrieren': 0.04591642794306938,
  'Möglichkeit': 0.15466927627649613,
  'Computer': 0.14510777250672308,
  'bieten': 0.27221442328719037,
  'so': 0.20911393211480495,
  '@card@': 0.6736895897891003,
  'durchführen': 0.04297579004775264,
  'Ziel':

In [52]:
def nearest_centroid(f,centers):
    similarities = [(c,cosinus(f,centers[c])) for c in centers]
    return sorted(similarities,key= lambda x:x[1],reverse=True)[0][0]

fs,c = test_data_tfidf[9]
nearest_centroid(fs,cls_centers)

'Fakultät I'

In [54]:
n = 0
correct = 0
for f,c in test_data_tfidf:
    p = nearest_centroid(f,cls_centers)
    n+= 1
    if p==c:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/float(n)))

69.7 Prozent korrekt


Das ist jetzt schon ziemlich gut. Schauen wir uns das Ergebnis jetzt etwas näner an:

In [56]:
gold = [c for f,c in test_data]
test = [nearest_centroid(f,cls_centers) for f,c in test_data_tfidf]
cm = nltk.ConfusionMatrix(gold, test)
print(cm) 

             |        F           F |
             |     F  a  F     F  a |
             |  F  a  k  a  F  a  k |
             |  a  k  u  k  a  k  u |
             |  k  u  l  u  k  u  l |
             |  u  l  t  l  u  l  t |
             |  l  t  ä  t  l  t  ä |
             |  t  ä  t  ä  t  ä  t |
             |  ä  t     t  ä  t    |
             |  t     I     t     V |
             |     I  I  I     V  I |
             |  I  I  I  V  V  I  I |
-------------+----------------------+
  Fakultät I |<18> .  .  .  .  .  2 |
 Fakultät II |  1<14> 4  1  .  .  . |
Fakultät III |  .  2<12> .  3  2  1 |
 Fakultät IV |  .  2  .<16> 2  .  . |
  Fakultät V |  1  .  .  5<10> 1  3 |
 Fakultät VI |  3  .  3  .  .<12> 2 |
Fakultät VII |  .  .  .  1  .  1<10>|
-------------+----------------------+
(row = reference; col = test)



# Aufgaben

1. Testen Sie den kNN Klassifikator für verschiedene Werte von k
2. Offenbar ist es schwierig zwischen F-IV und F-V zu unterscheiden. Welche Fachbereiche verstecken sich hinter diesen Nummern?
3. Schauen Sie sich einige Text, die falsch klassifiziert wurden an. Können Sie den Fehler erklären?